In [5]:
import os
import numpy as np
from pathlib import Path

# information
N_SUBJECTS = 339
N_PARCELS = 360
TR = 0.72  # Time resolution, in seconds
HEMIS = ["Right", "Left"]
N_RUNS = 2
HCP_DIR = "/home/amirreza/neuromatch/hcp_task"  # Update to your data path

EXPERIMENTS = {
    'MOTOR': {'runs': [5, 6], 'cond': ['lf', 'rf', 'lh', 'rh', 't', 'cue']},
    'WM': {'runs': [7, 8], 'cond': ['0bk_body', '0bk_faces', '0bk_places', '0bk_tools', '2bk_body', '2bk_faces', '2bk_places', '2bk_tools']},
    'EMOTION': {'runs': [9, 10], 'cond': ['fear', 'neut']},
    'GAMBLING': {'runs': [11, 12], 'cond': ['loss', 'win']},
    'LANGUAGE': {'runs': [13, 14], 'cond': ['math', 'story']},
    'RELATIONAL': {'runs': [15, 16], 'cond': ['match', 'relation']},
    'SOCIAL': {'runs': [17, 18], 'cond': ['mental', 'rnd']}
}


from pathlib import Path

def load_single_timeseries(subject, experiment, run, dir, remove_mean=True):
   """Load timeseries data for a single subject and single run.


   Args:
       subject (str): Subject ID to load
       experiment (str): Name of experiment
       run (int): 0-based run index (0 or 1 for the task)
       dir (str): Path to HCP directory
       remove_mean (bool): If True, subtract the parcel-wise mean


   Returns:
       ts (n_parcel x n_timepoint array): Array of BOLD data values
   """
   bold_run = EXPERIMENTS[experiment]['runs'][run]
   # Corrected path: Removed the extra "hcp_task"
   bold_path = os.path.join(dir, "subjects", str(subject), "timeseries")
   bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
   ts = np.load(os.path.join(bold_path, bold_file))
   if remove_mean:
       ts -= ts.mean(axis=1, keepdims=True)
   return ts

def load_evs(subject, experiment, run, dir):
   """Load EVs (explanatory variables) data for one task experiment.


   Args:
       subject (str): Subject ID to load
       experiment (str): Name of experiment
       run (int): 0-based run index (0 or 1 for the task)
       dir (str): Path to HCP directory


   Returns:
       evs (list of lists): A list of frames associated with each condition, clipped to valid timepoints
   """
   frames_list = []
   task_key = 'tfMRI_' + experiment + '_' + ['RL', 'LR'][run]
   # Load time series to get number of timepoints
   try:
       ts = load_single_timeseries(subject, experiment, run, dir, remove_mean=False)
       n_timepoints = ts.shape[1]
   except Exception as e:
       print(f"Error loading time series for {subject}/{experiment}/bold{EXPERIMENTS[experiment]['runs'][run]} to clip frames: {e}")
       return None


   for cond in EXPERIMENTS[experiment]['cond']:
       # Corrected path: Removed the extra "hcp_task"
       ev_file = os.path.join(dir, "subjects", str(subject), "EVs", task_key, f"{cond}.txt")
       try:
           ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
           ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
           # Determine when trial starts, rounded down
           start = np.floor(ev["onset"] / TR).astype(int)
           # Use trial duration to determine how many frames to include for trial
           duration = np.ceil(ev["duration"] / TR).astype(int)
           # Take the range of frames that correspond to this specific trial
           frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
           # Clip frames to valid timepoints
           clipped_frames = []
           for frame in frames:
               valid_frames = frame[frame < n_timepoints]
               if len(valid_frames) > 0:
                   clipped_frames.append(valid_frames)
           frames_list.append(clipped_frames if clipped_frames else [])
       except Exception as e:
           print(f"Error loading EV file {ev_file}: {e}")
           frames_list.append([])


   return frames_list

def concatenate_task_data(task, condition, dir):
   """
   Concatenate condition-specific frames from bold*.npy files for all subjects for a specific task and condition.
   Extract frames using load_evs, concatenate across runs, truncate to minimum frame count across subjects,
   and stack into a design matrix.

   Args:
       task (str): Task name (e.g., 'SOCIAL', 'WM')
       condition (str): Condition name (e.g., 'mental', '0bk_body')
       dir (str): Path to HCP directory

   Returns:
       task_cond_array (array): Array of shape (n_subjects, n_parcels, n_condition_timepoints)
                               or None if no data is concatenated
   """
   # Validate inputs
   if task not in EXPERIMENTS:
       print(f"Invalid task: {task}. Must be one of {list(EXPERIMENTS.keys())}.")
       return None
   if condition not in EXPERIMENTS[task]['cond']:
       print(f"Invalid condition: {condition} for task {task}. Must be one of {EXPERIMENTS[task]['cond']}.")
       return None

   # Load subjects
   subjects = [str(i) for i in range(339)]

   task_data = []
   frame_counts = []

   # Iterate through subjects
   for subject in subjects:
       subject_folder = Path(dir) / "subjects" / str(subject)
       if not subject_folder.is_dir():
           print(f"Subject directory {subject_folder} does not exist. Skipping.")
           continue

       # Load time series and EV frames for both runs
       subject_task_data = []
       subject_frames = []
       for run_idx in range(2):  # Two runs per task
           try:
               # Load time series
               ts = load_single_timeseries(subject, task, run_idx, dir, remove_mean=False)
               subject_task_data.append(ts)
               # Load EV frames
               frames_list = load_evs(subject, task, run_idx, dir)
               if frames_list is None:
                   raise Exception("EV frames are None")
               subject_frames.append(frames_list)
           except Exception as e:
               print(f"Error loading data or EVs for {subject}/{task}/bold{EXPERIMENTS[task]['runs'][run_idx]}: {e}")
               subject_task_data.append(None)
               subject_frames.append(None)

       # Check if any run failed to load
       if any(x is None for x in subject_task_data) or any(x is None for x in subject_frames):
           print(f"Skipping subject {subject} for task {task} due to missing data or EV frames.")
           continue

       # Get index of condition in EXPERIMENTS[task]['cond']
       cond_idx = EXPERIMENTS[task]['cond'].index(condition)

       try:
           # Concatenate frames for this condition across runs
           cond_frames = []
           for run_idx, run_frames in enumerate(subject_frames):
               # Flatten the frame list for this condition and adjust for run concatenation
               run_cond_frames = np.concatenate(run_frames[cond_idx]).astype(int) if run_frames[cond_idx] else np.array([])
               if run_idx == 1 and subject_task_data[0] is not None and len(run_cond_frames) > 0:
                   # Offset second run frames by the number of timepoints in first run
                   run_cond_frames += subject_task_data[0].shape[1]
               cond_frames.append(run_cond_frames)

           # Combine frames from both runs
           all_cond_frames = np.concatenate(cond_frames) if cond_frames and any(len(f) > 0 for f in cond_frames) else np.array([])

           # Skip if no valid frames
           if len(all_cond_frames) == 0:
               print(f"No valid frames for {subject}/{task}/{condition}. Skipping.")
               continue

           # Concatenate time series for both runs
           concatenated_ts = np.concatenate(subject_task_data, axis=1)

           # Ensure frames are within bounds
           max_timepoints = concatenated_ts.shape[1]
           valid_frames = all_cond_frames[all_cond_frames < max_timepoints]
           if len(valid_frames) == 0:
               print(f"No valid frames after clipping for {subject}/{task}/{condition}. Skipping.")
               continue

           # Extract condition-specific timepoints
           cond_ts = concatenated_ts[:, valid_frames]
           task_data.append(cond_ts)
           frame_counts.append(len(valid_frames))
       except Exception as e:
           print(f"Error processing condition {condition} for {subject}/{task}: {e}")
           continue

   # Stack subject arrays, truncating to minimum frame count
   if not task_data:
       print(f"No data concatenated for task {task}, condition {condition}.")
       return None
   try:
       # Find minimum number of frames across subjects
       min_frames = min(frame_counts) if frame_counts else 0
       if min_frames == 0:
           print(f"No valid frames for task {task}, condition {condition}.")
           return None

       # Truncate each subject's data to min_frames
       truncated_data = [data[:, :min_frames] for data in task_data]

       # Stack along a new axis (0) to get (n_subjects, n_parcels, n_condition_timepoints)
       task_cond_array = np.stack(truncated_data, axis=0)
       print(f"Task {task}, Condition {condition}: Concatenated array shape {task_cond_array.shape}")
       return task_cond_array
   except Exception as e:
       print(f"Error stacking subjects for task {task}, condition {condition}: {e}")
       return None

# Concat
task = "WM"
conditions = EXPERIMENTS[task]["cond"]
wm_data = {}
SAVE_DIR = "/home/amirreza/neuromatch/hcp_task/concatenated_data"
os.makedirs(SAVE_DIR, exist_ok=True)

for condition in conditions:
    print(f"Processing: {condition}")
    result = concatenate_task_data(task, condition, HCP_DIR)

    if result is not None:
        wm_data[condition] = result
        print(f"{condition} → shape: {result.shape}")

        # Save to disk right after creation
        save_path = os.path.join(SAVE_DIR, f"{condition}.npy")
        np.save(save_path, result)
        print(f"Saved: {save_path}")
    else:
        print(f"No data for: {condition}")

Processing: 0bk_body
Task WM, Condition 0bk_body: Concatenated array shape (339, 360, 78)
0bk_body → shape: (339, 360, 78)
Saved: /home/amirreza/neuromatch/hcp_task/concatenated_data/0bk_body.npy
Processing: 0bk_faces
Task WM, Condition 0bk_faces: Concatenated array shape (339, 360, 78)
0bk_faces → shape: (339, 360, 78)
Saved: /home/amirreza/neuromatch/hcp_task/concatenated_data/0bk_faces.npy
Processing: 0bk_places
Task WM, Condition 0bk_places: Concatenated array shape (339, 360, 78)
0bk_places → shape: (339, 360, 78)
Saved: /home/amirreza/neuromatch/hcp_task/concatenated_data/0bk_places.npy
Processing: 0bk_tools
Task WM, Condition 0bk_tools: Concatenated array shape (339, 360, 78)
0bk_tools → shape: (339, 360, 78)
Saved: /home/amirreza/neuromatch/hcp_task/concatenated_data/0bk_tools.npy
Processing: 2bk_body
Task WM, Condition 2bk_body: Concatenated array shape (339, 360, 78)
2bk_body → shape: (339, 360, 78)
Saved: /home/amirreza/neuromatch/hcp_task/concatenated_data/2bk_body.npy
Proc